In [279]:
#!pip install lightgbm
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [280]:
train = pd.read_csv('train_amex/train.csv')
test = pd.read_csv('test_LNMuIYp/test.csv')

In [281]:
historical = pd.read_csv('train_amex/historical_user_logs.csv')

In [282]:
submission = pd.read_csv('sample_submission_2s8l9nF.csv')
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128858 entries, 0 to 128857
Data columns (total 2 columns):
session_id    128858 non-null int64
is_click      128858 non-null int64
dtypes: int64(2)
memory usage: 2.0 MB


In [283]:
train.drop(columns=['product_category_2','session_id','DateTime','city_development_index','user_group_id'], axis=1, inplace=True)
test.drop(columns=['product_category_2','session_id','DateTime','city_development_index', 'user_group_id'], axis=1, inplace=True)
#historical.drop(columns=['DateTime'],axis=1,inplace=True)

In [284]:
#train_hist=pd.merge(train,historical, how='left')

In [285]:
#Since there are more 0s in is_click as compared to 1s. There are only 31331 1s and 431960 0s. 
#Need to take the equal amount of values.

#train_1 = train_raw[train_raw['is_click']==1]
#train_0 = train_raw[train_raw['is_click']==0].head(300000)

#train = train_1.append(train_0)

In [286]:
#train_hist.info()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463291 entries, 0 to 463290
Data columns (total 10 columns):
user_id               463291 non-null int64
product               463291 non-null object
campaign_id           463291 non-null int64
webpage_id            463291 non-null int64
product_category_1    463291 non-null int64
gender                445048 non-null object
age_level             445048 non-null float64
user_depth            445048 non-null float64
var_1                 463291 non-null int64
is_click              463291 non-null int64
dtypes: float64(2), int64(6), object(2)
memory usage: 35.3+ MB


In [287]:
#train_hist.head()
train.head()

,user_id,product,campaign_id,webpage_id,product_category_1,gender,age_level,user_depth,var_1,is_click
0,858557,C,359520,13787,4,Female,4.0,3.0,0,0
1,243253,C,105960,11085,5,Female,2.0,2.0,0,0
2,243253,C,359520,13787,4,Female,2.0,2.0,0,0
3,1097446,I,359520,13787,3,Male,3.0,3.0,1,0
4,663656,C,405490,60305,3,Male,2.0,3.0,1,0


In [288]:
#test_hist=pd.merge(test,historical, how='left', on = ['product', 'user_id'] )

In [289]:
#historical = historical.drop_duplicates()
#train_hist = train_hist.drop_duplicates()
#test_hist = test_hist.drop_duplicates()

In [290]:
#test_hist.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128858 entries, 0 to 128857
Data columns (total 9 columns):
user_id               128858 non-null int64
product               128858 non-null object
campaign_id           128858 non-null int64
webpage_id            128858 non-null int64
product_category_1    128858 non-null int64
gender                123174 non-null object
age_level             123174 non-null float64
user_depth            123174 non-null float64
var_1                 128858 non-null int64
dtypes: float64(2), int64(5), object(2)
memory usage: 8.8+ MB


In [291]:
#test_hist.head()
test.head()

,user_id,product,campaign_id,webpage_id,product_category_1,gender,age_level,user_depth,var_1
0,732573,J,404347,53587,1,Male,5.0,3.0,0
1,172910,I,118601,28529,3,NaN,NaN,NaN,1
2,172910,I,118601,28529,4,NaN,NaN,NaN,1
3,557318,G,118601,28529,5,Male,1.0,3.0,0
4,923896,H,118601,28529,5,Female,3.0,1.0,1


In [292]:
historical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24287534 entries, 0 to 24287533
Data columns (total 4 columns):
DateTime    object
user_id     int64
product     object
action      object
dtypes: int64(1), object(3)
memory usage: 741.2+ MB


In [293]:
historical.head()

,DateTime,user_id,product,action
0,2017-05-28 15:44,704,B,view
1,2017-05-29 07:08,499679,F,view
2,2017-05-29 07:10,499679,G,view
3,2017-05-29 07:10,499679,G,view
4,2017-05-29 07:10,499679,G,view


In [294]:
#train_hist.dtypes
train.dtypes

user_id                 int64
product                object
campaign_id             int64
webpage_id              int64
product_category_1      int64
gender                 object
age_level             float64
user_depth            float64
var_1                   int64
is_click                int64
dtype: object

In [295]:
train.groupby(['product'])['product'].count()

product
A     15391
B     22479
C    163501
D     41064
E     21452
F      7007
G      9414
H    109574
I     63711
J      9698
Name: product, dtype: int64

In [296]:
train = train.replace({'gender' : { 'Male' : 0, 'Female' : 1}})
test = test.replace({'gender' : { 'Male' : 0, 'Female' : 1}})

In [297]:
train = train.replace({'action' : { 'view' : 0, 'interest' : 1}})
test = test.replace({'action' : { 'view' : 0, 'interest' : 1}})

In [298]:
train=train.replace({'product': {'A': 0, 'B':1, 'C':2, "D":3,"E":4,"F":5,"G":6,"H":7,"I":8,"J":9}})
test=test.replace({'product': {'A': 0, 'B':1, 'C':2, "D":3,"E":4,"F":5,"G":6,"H":7,"I":8,"J":9}})

In [299]:
#converting datetime column into unix timestamp
#train_hist['DateTime']=pd.to_datetime(train_hist['DateTime'], format='%Y-%m-%d', utc=True)
#test_hist['DateTime']=pd.to_datetime(test_hist['DateTime'], format='%Y-%m-%d', utc=True)

In [300]:
train.groupby(['product_category_1'])['product_category_1'].count()

product_category_1
1     81141
2     69183
3    113812
4    133290
5     65865
Name: product_category_1, dtype: int64

In [301]:
#train_hist.groupby(['product_category_2'])['product_category_2'].count()

In [302]:
train.groupby(['gender'])['gender'].count()

gender
0.0    393454
1.0     51594
Name: gender, dtype: int64

In [303]:
train.groupby(['age_level'])['age_level'].count()

age_level
0.0       153
1.0     43367
2.0    143501
3.0    160581
4.0     63859
5.0     30828
6.0      2759
Name: age_level, dtype: int64

In [304]:
train.groupby(['user_depth'])['user_depth'].count()

user_depth
1.0     11861
2.0     30389
3.0    402798
Name: user_depth, dtype: int64

In [305]:
#train.groupby(['city_development_index'])['city_development_index'].count()

In [306]:
#train.groupby(['user_group_id'])['user_group_id'].count()

In [307]:
train.groupby(['var_1'])['var_1'].count()

var_1
0    267704
1    195587
Name: var_1, dtype: int64

In [308]:
train.groupby(['is_click'])['is_click'].count()

is_click
0    431960
1     31331
Name: is_click, dtype: int64

In [309]:
#train_hist= pd.merge(train,historical,how='inner',on=['user_id','product'])

In [310]:
#Imputing missing values

def missing_data_count(df):
  print(df.isna().sum())

missing_data_count(train)

user_id                   0
product                   0
campaign_id               0
webpage_id                0
product_category_1        0
gender                18243
age_level             18243
user_depth            18243
var_1                     0
is_click                  0
dtype: int64


In [311]:
missing_data_count(test)

user_id                  0
product                  0
campaign_id              0
webpage_id               0
product_category_1       0
gender                5684
age_level             5684
user_depth            5684
var_1                    0
dtype: int64


In [312]:
#test["city_development_index"] = test["city_development_index"].fillna(test['city_development_index'].mode()[0])
test["user_depth"] = test["user_depth"].fillna(test['user_depth'].mode()[0])
test["age_level"] = test["age_level"].fillna(test['age_level'].mode()[0])
test["gender"] = test["gender"].fillna(test['gender'].mode()[0])
#test["user_group_id"] = test["user_group_id"].fillna(test['user_group_id'].mode()[0])


#train["city_development_index"] = train["city_development_index"].fillna(train['city_development_index'].mode()[0])
train["user_depth"] = train["user_depth"].fillna(train['user_depth'].mode()[0])
train["age_level"] = train["age_level"].fillna(train['age_level'].mode()[0])
train["gender"] = train["gender"].fillna(train['gender'].mode()[0])
#train["user_group_id"] = train["user_group_id"].fillna(train['user_group_id'].mode()[0])

In [313]:
target = train['is_click']
target.value_counts()

train=train.drop(['is_click'],axis=1)
#test=test.drop(['product_category_2','session_id'],axis=1)

In [314]:
from sklearn.metrics import roc_auc_score
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb

In [315]:
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.33, random_state=42)

In [316]:
clf = lgb.LGBMClassifier(n_estimators=1000, silent=True, subsample=0.8, colsample_bytree=0.8, learning_rate=0.0075, scale_pos_weight=3.333)
clf.fit(X_train, y_train)
pred = clf.predict_proba(X_test)[:,1]
roc_auc_score(y_test, pred)

0.5831952038582853

In [317]:
clf = lgb.LGBMClassifier(n_estimators=1000, silent=True, subsample=0.8, colsample_bytree=0.8, learning_rate=0.0075, scale_pos_weight=3.333)
clf.fit(train, target)
pred = clf.predict_proba(test)[:,1]


In [318]:
print(pred.shape)
submission["is_click"].shape

(128858,)


(128858,)

In [319]:
submission["is_click"]=pred
submission.to_csv("amexpert_hack_solution_lgbm_with_equal_numbers.csv", index=False)

In [320]:
train.corr()

,user_id,product,campaign_id,webpage_id,product_category_1,gender,age_level,user_depth,var_1
user_id,1.000000,-0.010238,-0.009227,-0.022056,0.031165,0.029703,0.046416,-0.011420,0.057129
product,-0.010238,1.000000,-0.107040,-0.009801,-0.171914,0.008270,-0.022398,-0.004272,-0.041047
campaign_id,-0.009227,-0.107040,1.000000,0.606313,-0.113342,-0.001507,-0.061564,-0.022214,-0.006588
webpage_id,-0.022056,-0.009801,0.606313,1.000000,-0.280598,-0.043445,-0.116014,0.011444,-0.021678
product_category_1,0.031165,-0.171914,-0.113342,-0.280598,1.000000,0.034412,0.147104,0.000022,0.054029
gender,0.029703,0.008270,-0.001507,-0.043445,0.034412,1.000000,0.236647,-0.032465,-0.023761
age_level,0.046416,-0.022398,-0.061564,-0.116014,0.147104,0.236647,1.000000,0.024756,0.117223
user_depth,-0.011420,-0.004272,-0.022214,0.011444,0.000022,-0.032465,0.024756,1.000000,0.099676
var_1,0.057129,-0.041047,-0.006588,-0.021678,0.054029,-0.023761,0.117223,0.099676,1.000000
